In [1]:
import plotly.graph_objects as go
import numpy as np

# import time
import random
import scipy.optimize
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

/tmp/ipykernel_11471/637849031.py:9: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
import numpy as np

days = 20
sigma = 1
day = list(range(days + 1))
parking_demand = [0] * (days + 1)

scatters = []

while day[1] - day[0] > 1 / (24 * 60):
    print("day:", day)
    print("parking_demand:", parking_demand)
    print("sigma:", sigma)

    tmp = []
    tmp_day = []

    for i in range(len(parking_demand) - 1):
        tmp.append(parking_demand[i])
        tmp.append(
            0.5 * (parking_demand[i] + parking_demand[i + 1])
            + np.random.normal(0, sigma)
        )
        tmp_day.append(day[i])
        tmp_day.append(0.5 * (day[i] + day[i + 1]))

    tmp.append(parking_demand[-1])
    tmp_day.append(day[-1])

    parking_demand = tmp
    day = tmp_day
    sigma *= 0.5

    print("-----------------------------")


day: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
parking_demand: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
sigma: 1
-----------------------------
day: [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5, 16, 16.5, 17, 17.5, 18, 18.5, 19, 19.5, 20]
parking_demand: [0, -0.9877866811405285, 0, -0.30858455253995887, 0, -2.708414622077537, 0, -1.4041515826187865, 0, 0.5416329731152082, 0, 0.30909513521003085, 0, 0.5273341532217027, 0, 1.1997242330068683, 0, -0.9800550976292908, 0, -0.28803090827722005, 0, -0.9047201197415325, 0, -0.027305657808171072, 0, 0.43011574572622513, 0, 1.3290448549884215, 0, -0.5887580251639162, 0, 0.5166092009523219, 0, 0.2668884090117403, 0, -2.6486519115517355, 0, 2.8997608744772627, 0, 1.9629989644252472, 0]
sigma: 0.5
-----------------------------
day: [0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25, 

In [3]:
day = np.array(day)
parking_demand = np.array(parking_demand)

In [4]:
# Some random function for demand over time
parking_demand += 2  # Some constant demand
# Some periodic demand change
parking_demand += 7.25 * (np.sin(day * 2 * np.pi) + 1)
# Cant have negative demand...
parking_demand = np.clip(parking_demand, 0, 1e9)


In [5]:
len(parking_demand)

40961

In [6]:
# Lets say that we only have X spots
num_spots = 12
# Cant have more parked than spots. People also come as whole numbers.
parking_used = np.clip(np.round(parking_demand), 0, num_spots)

# Sample observations
num_samples = min([120, len(day)])  # maximum number of samples to be selected.

sample_ind = sorted(
    random.sample(range(len(day)), num_samples)
)  # sorted list of samples

sample_day = day[sample_ind]  # fetch day values as per samples

optimal_sample_used = parking_used[sample_ind]  # orignal sample points

sample_parking_used = parking_used[
    sample_ind
]  # fetch corresponding parking usage for sampled days
# Add noise to the observations
for i, sample in enumerate(sample_parking_used):
    offset = random.choice([-2, -1, 0, 1, 2])  # adding noise by values in list
    sample_parking_used[i] += offset


In [22]:
sample_parking_used

array([12., 14.,  5.,  2.,  0.,  0.,  8., 11., 14., 10.,  7.,  6.,  7.,
       12., 12., 14.,  9.,  3.,  9.,  4.,  3., 13.,  8.,  9.,  0., 10.,
       10., 14., 12.,  5.,  4.,  2., 11., 11., 12.,  7.,  5.,  0.,  6.,
        5.,  9., 11., 10., 11.,  3.,  3.,  7., 14., 13.,  0., 11., 13.,
       13., 14., 11., 11.,  4.,  4.,  5.,  6.,  8., 10., 11., 11., 11.,
        2.,  7., 12., 10., 11., 11., 13., 12., 13., 10.,  5.,  5.,  2.,
        4.,  6.,  7.,  8., 11., 12., 12.,  5.,  4.,  4., 11., 11., 14.,
       11.,  3.,  2.,  5.,  8., 10.,  2.,  2.,  4.,  3.,  4., 11., 11.,
        7.,  4.,  4.,  9., 12., 10., 11., 11.,  4.,  8.,  9., 12., 10.,
       12.,  7.,  9.])

In [7]:
# Scatter plot of sample observations

usage_scatter = go.Scatter(
    x=day, y=parking_used, line_width=2, line_color="Red", name="Spot usage"
)

obs_scatter = go.Scatter(
    x=sample_day,
    y=sample_parking_used,
    mode="markers",
    marker_symbol="star",
    marker_line_width=2,
    marker_size=8,
    name="Noisy observation points",
)

# Scatter plot of optimal sample observations
optimal_obs_scatter = go.Scatter(
    x=sample_day,
    y=optimal_sample_used,
    mode="markers",
    marker_line_width=2,
    marker_size=8,
    name="Observation points",
)

# Set up drawing
layout = go.Layout(
    title="parking",
    width=1800,
    height=1000,
    yaxis_title="Number of cars",
    margin=dict(l=80, r=80, t=100, b=99),
    xaxis_title="Day",
    xaxis_dtick=1,
    yaxis_dtick=1,
    xaxis_range=[0, days],
    yaxis_range=[-0.1, 18.5],
    showlegend=True,
)
layout.update(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))

# Scatter plot of sample observations
layout.title.text = "Parking usage observations over time"
fig_obs = go.Figure(
    data=[usage_scatter, obs_scatter, optimal_obs_scatter], layout=layout
)
fig_obs.show()


## Gaussian Processes 

In [8]:
sample_day.reshape(-1, 1)


array([[ 0.15820312],
       [ 0.328125  ],
       [ 0.60644531],
       [ 0.6640625 ],
       [ 0.70361328],
       [ 1.73681641],
       [ 2.03125   ],
       [ 2.03808594],
       [ 2.09619141],
       [ 2.19482422],
       [ 2.45849609],
       [ 2.90576172],
       [ 2.93847656],
       [ 3.32519531],
       [ 3.34179688],
       [ 3.41162109],
       [ 3.44091797],
       [ 3.703125  ],
       [ 4.01806641],
       [ 4.77929688],
       [ 4.88574219],
       [ 5.43359375],
       [ 5.48095703],
       [ 5.48779297],
       [ 5.71191406],
       [ 6.12744141],
       [ 6.19970703],
       [ 6.33154297],
       [ 6.46533203],
       [ 6.64013672],
       [ 6.67138672],
       [ 6.79150391],
       [ 7.11230469],
       [ 7.34521484],
       [ 7.4140625 ],
       [ 7.58203125],
       [ 7.62304688],
       [ 7.71679688],
       [ 7.84667969],
       [ 7.95263672],
       [ 8.04296875],
       [ 8.17138672],
       [ 8.24316406],
       [ 8.4375    ],
       [ 8.66210938],
       [ 8

In [9]:
sample_parking_used

array([12., 14.,  5.,  2.,  0.,  0.,  8., 11., 14., 10.,  7.,  6.,  7.,
       12., 12., 14.,  9.,  3.,  9.,  4.,  3., 13.,  8.,  9.,  0., 10.,
       10., 14., 12.,  5.,  4.,  2., 11., 11., 12.,  7.,  5.,  0.,  6.,
        5.,  9., 11., 10., 11.,  3.,  3.,  7., 14., 13.,  0., 11., 13.,
       13., 14., 11., 11.,  4.,  4.,  5.,  6.,  8., 10., 11., 11., 11.,
        2.,  7., 12., 10., 11., 11., 13., 12., 13., 10.,  5.,  5.,  2.,
        4.,  6.,  7.,  8., 11., 12., 12.,  5.,  4.,  4., 11., 11., 14.,
       11.,  3.,  2.,  5.,  8., 10.,  2.,  2.,  4.,  3.,  4., 11., 11.,
        7.,  4.,  4.,  9., 12., 10., 11., 11.,  4.,  8.,  9., 12., 10.,
       12.,  7.,  9.])

In [47]:
filtered_sample_parking_used = sample_parking_used[sample_parking_used < 12]
filtered_sample_parking_used.reshape(-1, 1)


array([[ 5.],
       [ 2.],
       [ 0.],
       [ 0.],
       [ 8.],
       [11.],
       [10.],
       [ 7.],
       [ 6.],
       [ 7.],
       [ 9.],
       [ 3.],
       [ 9.],
       [ 4.],
       [ 3.],
       [ 8.],
       [ 9.],
       [ 0.],
       [10.],
       [10.],
       [ 5.],
       [ 4.],
       [ 2.],
       [11.],
       [11.],
       [ 7.],
       [ 5.],
       [ 0.],
       [ 6.],
       [ 5.],
       [ 9.],
       [11.],
       [10.],
       [11.],
       [ 3.],
       [ 3.],
       [ 7.],
       [ 0.],
       [11.],
       [11.],
       [11.],
       [ 4.],
       [ 4.],
       [ 5.],
       [ 6.],
       [ 8.],
       [10.],
       [11.],
       [11.],
       [11.],
       [ 2.],
       [ 7.],
       [10.],
       [11.],
       [11.],
       [10.],
       [ 5.],
       [ 5.],
       [ 2.],
       [ 4.],
       [ 6.],
       [ 7.],
       [ 8.],
       [11.],
       [ 5.],
       [ 4.],
       [ 4.],
       [11.],
       [11.],
       [11.],
       [ 3.],
      

In [48]:
x_train = sample_day.reshape(-1, 1)
y_train = filtered_sample_parking_used
num_test_samples = 120
X_test = np.linspace(day[0], day[-1], num=num_test_samples).reshape(-1, 1)


In [49]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import RBF

# Define the kernel for GPR
kernel_rq = RationalQuadratic(length_scale=1.0, alpha=1.0)  # Rational Quadratic kernel
kernel_rbf = RBF(length_scale=1.0)

In [50]:
def gpr_train(kernel, x_train, y_train):
    # Create the GPR model
    gpr = GaussianProcessRegressor(kernel=kernel)
    # Fit the GPR model to the training data
    gpr.fit(x_train, y_train)

    # Perform predictions using the trained GPR model
    y_pred, y_std = gpr.predict(X_test, return_std=True)
    # y_pred: Predicted target values
    # y_std: Standard deviation of predictions

    # Access learned model properties
    kernel_params = gpr.kernel_  # Learned kernel parameters
    noise_level = gpr.alpha_  # Estimated noise level (if available)

    # Print or analyze the results as needed
    print("Predicted values:", y_pred)
    print("Prediction uncertainty (std):", y_std)
    print("Learned kernel parameters:", kernel_params)
    print("Estimated noise level:", noise_level)

    return x_train, X_test, y_train, y_pred, y_std


In [51]:
def viz_gpr_results(x_train, X_test, y_train, y_pred, y_std):
    # Create a scatter trace for training data
    train_trace = go.Scatter(
        x=x_train.squeeze(),
        y=y_train,
        mode="markers",
        name="Training Data",
        marker=dict(color="blue"),
    )

    # Create a line trace for predicted values
    pred_trace = go.Scatter(
        x=X_test.squeeze(),
        y=y_pred,
        mode="lines",
        name="Predicted Values",
        line=dict(color="red"),
    )

    # Create a shaded region for uncertainty
    uncertainty_trace = go.Scatter(
        x=X_test.squeeze(),
        y=y_pred + 2 * y_std,
        mode="lines",
        name="Uncertainty",
        line=dict(color="orange", width=0),
        fill="tonexty",
        fillcolor="rgba(255, 165, 0, 0.3)",
    )

    # Create the figure layout
    layout = go.Layout(
        title="GPR Results", xaxis=dict(title="Day"), yaxis=dict(title="Parking Usage")
    )

    # Create the figure and add traces
    fig = go.Figure(data=[train_trace, pred_trace, uncertainty_trace], layout=layout)

    # Show the figure
    fig.show()


In [52]:
# Create the GPR model
gpr = GaussianProcessRegressor(kernel=kernel_rq)
# Fit the GPR model to the training data
gpr.fit(x_train, y_train)

# Perform predictions using the trained GPR model
y_pred, y_std = gpr.predict(X_test, return_std=True)
# y_pred: Predicted target values
# y_std: Standard deviation of predictions

# Access learned model properties
kernel_params = gpr.kernel_  # Learned kernel parameters
noise_level = gpr.alpha_  # Estimated noise level (if available)

# Print or analyze the results as needed
# print("Predicted values:", y_pred)
# print("Prediction uncertainty (std):", y_std)
# print("Learned kernel parameters:", kernel_params)
print("Estimated noise level:", noise_level)


ValueError: Found input variables with inconsistent numbers of samples: [120, 95]

In [35]:
# Create a scatter trace for training data
train_trace = go.Scatter(
    x=x_train.squeeze(),
    y=y_train,
    mode="markers",
    name="Training Data",
    marker=dict(color="blue"),
)

# Create a line trace for predicted values
pred_trace = go.Scatter(
    x=X_test.squeeze(),
    y=y_pred,
    mode="lines",
    name="Predicted Values",
    line=dict(color="red"),
)

# Create a shaded region for uncertainty
uncertainty_trace = go.Scatter(
    x=X_test.squeeze(),
    y=y_pred + 2 * y_std,
    mode="lines",
    name="Uncertainty",
    line=dict(color="orange", width=0),
    fill="tonexty",
    fillcolor="rgba(255, 165, 0, 0.3)",
)

# Create the figure layout
layout = go.Layout(
    title="GPR Results", xaxis=dict(title="Day"), yaxis=dict(title="Parking Usage")
)

# Create the figure and add traces
fig = go.Figure(data=[train_trace, pred_trace, uncertainty_trace], layout=layout)

# Show the figure
fig.show()


In [36]:
# Create a scatter trace for training data
train_trace = go.Scatter(
    x=x_train.squeeze(),
    y=y_train,
    mode="markers",
    name="Training Data",
    marker=dict(color="blue"),
)

# Create a line trace for predicted values
pred_trace = go.Scatter(
    x=X_test.squeeze(),
    y=y_pred,
    mode="lines",
    name="Predicted Values",
    line=dict(color="red"),
)

# Create a shaded region for uncertainty
uncertainty_trace = go.Scatter(
    x=X_test.squeeze(),
    y=y_pred + 2 * y_std,
    mode="lines",
    name="Uncertainty",
    line=dict(color="orange", width=0),
    fill="tonexty",
    fillcolor="rgba(255, 165, 0, 0.3)",
)

# Create the figure layout
layout = go.Layout(
    title="GPR Results", xaxis=dict(title="Day"), yaxis=dict(title="Parking Usage")
)

# Create the figure and add traces
fig = go.Figure(data=[train_trace, pred_trace, uncertainty_trace], layout=layout)

# Show the figure
fig.show()


In [53]:
# combined kernel

from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, ConstantKernel, Sum

l = 2
l1 = 1
l2 = 1
w1 = 1
w2 = 2
a = b = c = d = 1
# Define the individual kernels
rbf_kernel = RBF(length_scale=l)
sine_kernel_1 = ExpSineSquared(length_scale=l1, periodicity=w1)
sine_kernel_2 = ExpSineSquared(length_scale=l2, periodicity=w2)
linear_kernel = ConstantKernel(constant_value=c) * ConstantKernel(constant_value=d)

# Define the composite kernel as a sum of individual kernels with respective weights
composite_kernel = (
    a * rbf_kernel + b * sine_kernel_1 + c * sine_kernel_2 + linear_kernel
)


In [54]:
x_train, X_test, y_train, y_pred, y_std = gpr_train(composite_kernel)

TypeError: gpr_train() missing 2 required positional arguments: 'x_train' and 'y_train'

In [55]:
viz_gpr_results(x_train, X_test, y_train, y_pred, y_std)

In [ ]:
# avg parallel parking size l = 7.924 m, w= 2.438m
# acc to stockholms stad, avg space len is 6m


## Calucuate parking capacity

In [ ]:
file = "./parking_count.csv"
parking_gdf = gpd.read_file(
    file, GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO"
)


In [ ]:
type(parking_gdf)

geopandas.geodataframe.GeoDataFrame

In [ ]:
parking_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   FID                   117 non-null    object  
 1   index                 117 non-null    object  
 2   Unnamed: 0            117 non-null    object  
 3   id                    117 non-null    object  
 4   x                     117 non-null    object  
 5   y                     117 non-null    object  
 6   z                     117 non-null    object  
 7   first_timestamp       117 non-null    object  
 8   last_timestamp        117 non-null    object  
 9   latitude              117 non-null    object  
 10  longitude             117 non-null    object  
 11  first_unix_timestamp  117 non-null    object  
 12  last_unix_timestamp   117 non-null    object  
 13  x_3011                117 non-null    object  
 14  y_3011                117 non-null    object  
 15

In [ ]:
# parking_gdf.drop(columns=['geometry','Unnamed: 0'],axis=1, inplace = True)

In [ ]:
type(parking_gdf["closest_parking_geom"].iloc[0])


str

In [ ]:
parking_gdf["closest_parking_geom"].iloc[0]


'[[151574.473349, 6576963.887167], [151589.196784, 6576952.894341]]'

In [ ]:
import ast

parking_gdf["closest_parking_geom"] = parking_gdf["closest_parking_geom"].apply(
    lambda x: eval(x)
)


In [ ]:
parking_gdf[""]
